In [24]:
import requests
import pandas as pd
import statistics as st
import numpy as np

In [2]:
# import path for imports
import sys
import os
import_path = os.path.join('c:', os.sep, "Users", "shane", "OneDrive", "MyDocuments", "AppDev", "my_imports")
sys.path.insert(1, import_path)

In [3]:
# import token
from config_keys import ncdc

In [5]:
# get state data from NOAA NCDC
base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/locations"
token = ncdc
params = {
    'locationcategoryid' : 'ST',
    'startdate' : '2000-01-01',
    'enddate' : '2010-01-01',
    'limit' : 51
}

results = requests.get(base_url, params=params, headers={'token':token})

In [7]:
weather_states = results.json()['results']

In [45]:
def get_value(state, measure, month, year):
    """function to return an average value for a measurement"""
    base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data?"
    
    params = {
    'datasetid' : 'GSOM',
    'locationid' : state,
    'units' : 'standard',
    'datatypeid' : measure
    }
    
    # we need a 2 digit month
    if month < 10:
        params['startdate'] = f'{year}-0{month}-01'
        params['enddate'] = f'{year}-0{month}-01'
    else:
        params['startdate'] = f'{year}-{month}-01'
        params['enddate'] = f'{year}-{month}-01'
        
    #print(params)
    try:
        results = requests.get(base_url, params=params, headers={'token':token})
    
    # exception list from https://www.secopshub.com/t/handling-api-errors-using-python-requests/589
    except requests.exceptions.HTTPError as errh:
        print("An Http Error occurred:" + repr(errh))
        return(False)
    except requests.exceptions.ConnectionError as errc:
        print("An Error Connecting to the API occurred:" + repr(errc))
        return(False)
    except requests.exceptions.Timeout as errt:
        print("A Timeout Error occurred:" + repr(errt))
        return(False)
    except requests.exceptions.RequestException as err:
        print("An Unknown Error occurred" + repr(err))
        return(False)
    
    try:
        json_result = results.json()
    except:
        print("No JSON data")
        return(False)
    
    try:
        response_results = json_result['results']
        return(False)
    except KeyError:
        print('No results in json')
        return(False)
        
    measures = []
    for result in response_results:
        try:
            measures.append(result['value'])
        except KeyError:
            print("no value was found in results")
            return(False)

    return(st.mean(measures))
        

In [36]:
# create dictionary of lists for the datframe
# Get avergage monthly temperature data
# TAVG measure from GSOM is the 
weather_data = {
    'Year' : [],
    'Month' : [],
    'State' : [],
    'TAVG' : []
}

measurements = ['TAVG']
start_year = 2000
end_year = 2010

# build a list to capture the failures to try again if needed
faliures = []

for state in weather_states:
    #print(state['id'])
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            for measure in measurements:
                print(f'{state["id"]} - {state["name"]} - {month} - {year} - {measure}')
                
                value = get_value(state["id"], measure, month, year)
                
                if value:
                    weather_data[measure].append(value)
                    weather_data['State'].append(state["name"])
                    weather_data['Year'].append(year)
                    weather_data['Month'].append(month)
                else:
                    failures.append([state["id"], measure, month, year])


FIPS:01 - Alabama - 1 - 2000 - TAVG
FIPS:01 - Alabama - 2 - 2000 - TAVG
FIPS:01 - Alabama - 3 - 2000 - TAVG
FIPS:01 - Alabama - 4 - 2000 - TAVG
FIPS:01 - Alabama - 5 - 2000 - TAVG
FIPS:01 - Alabama - 6 - 2000 - TAVG
FIPS:01 - Alabama - 7 - 2000 - TAVG
FIPS:01 - Alabama - 8 - 2000 - TAVG
FIPS:01 - Alabama - 9 - 2000 - TAVG
FIPS:01 - Alabama - 10 - 2000 - TAVG
FIPS:01 - Alabama - 11 - 2000 - TAVG
FIPS:01 - Alabama - 12 - 2000 - TAVG
FIPS:01 - Alabama - 1 - 2001 - TAVG
FIPS:01 - Alabama - 2 - 2001 - TAVG
FIPS:01 - Alabama - 3 - 2001 - TAVG
FIPS:01 - Alabama - 4 - 2001 - TAVG
FIPS:01 - Alabama - 5 - 2001 - TAVG
FIPS:01 - Alabama - 6 - 2001 - TAVG
FIPS:01 - Alabama - 7 - 2001 - TAVG
FIPS:01 - Alabama - 8 - 2001 - TAVG
FIPS:01 - Alabama - 9 - 2001 - TAVG
FIPS:01 - Alabama - 10 - 2001 - TAVG
FIPS:01 - Alabama - 11 - 2001 - TAVG
FIPS:01 - Alabama - 12 - 2001 - TAVG
FIPS:01 - Alabama - 1 - 2002 - TAVG
FIPS:01 - Alabama - 2 - 2002 - TAVG
FIPS:01 - Alabama - 3 - 2002 - TAVG
FIPS:01 - Alabama - 4 

FIPS:02 - Alaska - 2 - 2008 - TAVG
FIPS:02 - Alaska - 3 - 2008 - TAVG
FIPS:02 - Alaska - 4 - 2008 - TAVG
FIPS:02 - Alaska - 5 - 2008 - TAVG
FIPS:02 - Alaska - 6 - 2008 - TAVG
FIPS:02 - Alaska - 7 - 2008 - TAVG
FIPS:02 - Alaska - 8 - 2008 - TAVG
FIPS:02 - Alaska - 9 - 2008 - TAVG
FIPS:02 - Alaska - 10 - 2008 - TAVG
FIPS:02 - Alaska - 11 - 2008 - TAVG
FIPS:02 - Alaska - 12 - 2008 - TAVG
FIPS:02 - Alaska - 1 - 2009 - TAVG
FIPS:02 - Alaska - 2 - 2009 - TAVG
FIPS:02 - Alaska - 3 - 2009 - TAVG
FIPS:02 - Alaska - 4 - 2009 - TAVG
FIPS:02 - Alaska - 5 - 2009 - TAVG
FIPS:02 - Alaska - 6 - 2009 - TAVG
FIPS:02 - Alaska - 7 - 2009 - TAVG
FIPS:02 - Alaska - 8 - 2009 - TAVG
FIPS:02 - Alaska - 9 - 2009 - TAVG
FIPS:02 - Alaska - 10 - 2009 - TAVG
FIPS:02 - Alaska - 11 - 2009 - TAVG
FIPS:02 - Alaska - 12 - 2009 - TAVG
FIPS:02 - Alaska - 1 - 2010 - TAVG
FIPS:02 - Alaska - 2 - 2010 - TAVG
FIPS:02 - Alaska - 3 - 2010 - TAVG
FIPS:02 - Alaska - 4 - 2010 - TAVG
FIPS:02 - Alaska - 5 - 2010 - TAVG
FIPS:02 - Alas

FIPS:05 - Arkansas - 12 - 2004 - TAVG
FIPS:05 - Arkansas - 1 - 2005 - TAVG
FIPS:05 - Arkansas - 2 - 2005 - TAVG
FIPS:05 - Arkansas - 3 - 2005 - TAVG
FIPS:05 - Arkansas - 4 - 2005 - TAVG
FIPS:05 - Arkansas - 5 - 2005 - TAVG
FIPS:05 - Arkansas - 6 - 2005 - TAVG
FIPS:05 - Arkansas - 7 - 2005 - TAVG
FIPS:05 - Arkansas - 8 - 2005 - TAVG
FIPS:05 - Arkansas - 9 - 2005 - TAVG
FIPS:05 - Arkansas - 10 - 2005 - TAVG
FIPS:05 - Arkansas - 11 - 2005 - TAVG
FIPS:05 - Arkansas - 12 - 2005 - TAVG
FIPS:05 - Arkansas - 1 - 2006 - TAVG
FIPS:05 - Arkansas - 2 - 2006 - TAVG
FIPS:05 - Arkansas - 3 - 2006 - TAVG
FIPS:05 - Arkansas - 4 - 2006 - TAVG
FIPS:05 - Arkansas - 5 - 2006 - TAVG
FIPS:05 - Arkansas - 6 - 2006 - TAVG
FIPS:05 - Arkansas - 7 - 2006 - TAVG
FIPS:05 - Arkansas - 8 - 2006 - TAVG
FIPS:05 - Arkansas - 9 - 2006 - TAVG
FIPS:05 - Arkansas - 10 - 2006 - TAVG
FIPS:05 - Arkansas - 11 - 2006 - TAVG
FIPS:05 - Arkansas - 12 - 2006 - TAVG
FIPS:05 - Arkansas - 1 - 2007 - TAVG
FIPS:05 - Arkansas - 2 - 2007 -

FIPS:08 - Colorado - 9 - 2000 - TAVG
FIPS:08 - Colorado - 10 - 2000 - TAVG
FIPS:08 - Colorado - 11 - 2000 - TAVG
FIPS:08 - Colorado - 12 - 2000 - TAVG
FIPS:08 - Colorado - 1 - 2001 - TAVG
FIPS:08 - Colorado - 2 - 2001 - TAVG
FIPS:08 - Colorado - 3 - 2001 - TAVG
FIPS:08 - Colorado - 4 - 2001 - TAVG
FIPS:08 - Colorado - 5 - 2001 - TAVG
FIPS:08 - Colorado - 6 - 2001 - TAVG
FIPS:08 - Colorado - 7 - 2001 - TAVG
FIPS:08 - Colorado - 8 - 2001 - TAVG
FIPS:08 - Colorado - 9 - 2001 - TAVG
FIPS:08 - Colorado - 10 - 2001 - TAVG
FIPS:08 - Colorado - 11 - 2001 - TAVG
FIPS:08 - Colorado - 12 - 2001 - TAVG
FIPS:08 - Colorado - 1 - 2002 - TAVG
FIPS:08 - Colorado - 2 - 2002 - TAVG
FIPS:08 - Colorado - 3 - 2002 - TAVG
FIPS:08 - Colorado - 4 - 2002 - TAVG
FIPS:08 - Colorado - 5 - 2002 - TAVG
FIPS:08 - Colorado - 6 - 2002 - TAVG
FIPS:08 - Colorado - 7 - 2002 - TAVG
FIPS:08 - Colorado - 8 - 2002 - TAVG
FIPS:08 - Colorado - 9 - 2002 - TAVG
FIPS:08 - Colorado - 10 - 2002 - TAVG
FIPS:08 - Colorado - 11 - 2002 

FIPS:09 - Connecticut - 6 - 2007 - TAVG
FIPS:09 - Connecticut - 7 - 2007 - TAVG
FIPS:09 - Connecticut - 8 - 2007 - TAVG
FIPS:09 - Connecticut - 9 - 2007 - TAVG
FIPS:09 - Connecticut - 10 - 2007 - TAVG
FIPS:09 - Connecticut - 11 - 2007 - TAVG
FIPS:09 - Connecticut - 12 - 2007 - TAVG
FIPS:09 - Connecticut - 1 - 2008 - TAVG
FIPS:09 - Connecticut - 2 - 2008 - TAVG
FIPS:09 - Connecticut - 3 - 2008 - TAVG
FIPS:09 - Connecticut - 4 - 2008 - TAVG
FIPS:09 - Connecticut - 5 - 2008 - TAVG
FIPS:09 - Connecticut - 6 - 2008 - TAVG
FIPS:09 - Connecticut - 7 - 2008 - TAVG
FIPS:09 - Connecticut - 8 - 2008 - TAVG
FIPS:09 - Connecticut - 9 - 2008 - TAVG
FIPS:09 - Connecticut - 10 - 2008 - TAVG
FIPS:09 - Connecticut - 11 - 2008 - TAVG
FIPS:09 - Connecticut - 12 - 2008 - TAVG
FIPS:09 - Connecticut - 1 - 2009 - TAVG
FIPS:09 - Connecticut - 2 - 2009 - TAVG
FIPS:09 - Connecticut - 3 - 2009 - TAVG
FIPS:09 - Connecticut - 4 - 2009 - TAVG
FIPS:09 - Connecticut - 5 - 2009 - TAVG
FIPS:09 - Connecticut - 6 - 2009 -

...

FIPS:49 - Utah - 12 - 2002 - TAVG
FIPS:49 - Utah - 1 - 2003 - TAVG
FIPS:49 - Utah - 2 - 2003 - TAVG
FIPS:49 - Utah - 3 - 2003 - TAVG
FIPS:49 - Utah - 4 - 2003 - TAVG
FIPS:49 - Utah - 5 - 2003 - TAVG
FIPS:49 - Utah - 6 - 2003 - TAVG
FIPS:49 - Utah - 7 - 2003 - TAVG
FIPS:49 - Utah - 8 - 2003 - TAVG
FIPS:49 - Utah - 9 - 2003 - TAVG
FIPS:49 - Utah - 10 - 2003 - TAVG
FIPS:49 - Utah - 11 - 2003 - TAVG
FIPS:49 - Utah - 12 - 2003 - TAVG
FIPS:49 - Utah - 1 - 2004 - TAVG
FIPS:49 - Utah - 2 - 2004 - TAVG
FIPS:49 - Utah - 3 - 2004 - TAVG
FIPS:49 - Utah - 4 - 2004 - TAVG
FIPS:49 - Utah - 5 - 2004 - TAVG
FIPS:49 - Utah - 6 - 2004 - TAVG
FIPS:49 - Utah - 7 - 2004 - TAVG
FIPS:49 - Utah - 8 - 2004 - TAVG
FIPS:49 - Utah - 9 - 2004 - TAVG
FIPS:49 - Utah - 10 - 2004 - TAVG
FIPS:49 - Utah - 11 - 2004 - TAVG
FIPS:49 - Utah - 12 - 2004 - TAVG
FIPS:49 - Utah - 1 - 2005 - TAVG
FIPS:49 - Utah - 2 - 2005 - TAVG
FIPS:49 - Utah - 3 - 2005 - TAVG
FIPS:49 - Utah - 4 - 2005 - TAVG
FIPS:49 - Utah - 5 - 2005 - TAVG
FIP

FIPS:51 - Virginia - 6 - 2000 - TAVG
FIPS:51 - Virginia - 7 - 2000 - TAVG
FIPS:51 - Virginia - 8 - 2000 - TAVG
FIPS:51 - Virginia - 9 - 2000 - TAVG
FIPS:51 - Virginia - 10 - 2000 - TAVG
FIPS:51 - Virginia - 11 - 2000 - TAVG
FIPS:51 - Virginia - 12 - 2000 - TAVG
FIPS:51 - Virginia - 1 - 2001 - TAVG
FIPS:51 - Virginia - 2 - 2001 - TAVG
FIPS:51 - Virginia - 3 - 2001 - TAVG
FIPS:51 - Virginia - 4 - 2001 - TAVG
FIPS:51 - Virginia - 5 - 2001 - TAVG
FIPS:51 - Virginia - 6 - 2001 - TAVG
FIPS:51 - Virginia - 7 - 2001 - TAVG
FIPS:51 - Virginia - 8 - 2001 - TAVG
FIPS:51 - Virginia - 9 - 2001 - TAVG
FIPS:51 - Virginia - 10 - 2001 - TAVG
FIPS:51 - Virginia - 11 - 2001 - TAVG
FIPS:51 - Virginia - 12 - 2001 - TAVG
FIPS:51 - Virginia - 1 - 2002 - TAVG
FIPS:51 - Virginia - 2 - 2002 - TAVG
FIPS:51 - Virginia - 3 - 2002 - TAVG
FIPS:51 - Virginia - 4 - 2002 - TAVG
FIPS:51 - Virginia - 5 - 2002 - TAVG
FIPS:51 - Virginia - 6 - 2002 - TAVG
FIPS:51 - Virginia - 7 - 2002 - TAVG
FIPS:51 - Virginia - 8 - 2002 - 

FIPS:53 - Washington - 6 - 2007 - TAVG
FIPS:53 - Washington - 7 - 2007 - TAVG
FIPS:53 - Washington - 8 - 2007 - TAVG
FIPS:53 - Washington - 9 - 2007 - TAVG
FIPS:53 - Washington - 10 - 2007 - TAVG
FIPS:53 - Washington - 11 - 2007 - TAVG
FIPS:53 - Washington - 12 - 2007 - TAVG
FIPS:53 - Washington - 1 - 2008 - TAVG
FIPS:53 - Washington - 2 - 2008 - TAVG
FIPS:53 - Washington - 3 - 2008 - TAVG
FIPS:53 - Washington - 4 - 2008 - TAVG
FIPS:53 - Washington - 5 - 2008 - TAVG
FIPS:53 - Washington - 6 - 2008 - TAVG
FIPS:53 - Washington - 7 - 2008 - TAVG
FIPS:53 - Washington - 8 - 2008 - TAVG
FIPS:53 - Washington - 9 - 2008 - TAVG
FIPS:53 - Washington - 10 - 2008 - TAVG
FIPS:53 - Washington - 11 - 2008 - TAVG
FIPS:53 - Washington - 12 - 2008 - TAVG
FIPS:53 - Washington - 1 - 2009 - TAVG
FIPS:53 - Washington - 2 - 2009 - TAVG
FIPS:53 - Washington - 3 - 2009 - TAVG
FIPS:53 - Washington - 4 - 2009 - TAVG
FIPS:53 - Washington - 5 - 2009 - TAVG
FIPS:53 - Washington - 6 - 2009 - TAVG
FIPS:53 - Washingto

FIPS:55 - Wisconsin - 2 - 2002 - TAVG
FIPS:55 - Wisconsin - 3 - 2002 - TAVG
FIPS:55 - Wisconsin - 4 - 2002 - TAVG
FIPS:55 - Wisconsin - 5 - 2002 - TAVG
FIPS:55 - Wisconsin - 6 - 2002 - TAVG
FIPS:55 - Wisconsin - 7 - 2002 - TAVG
FIPS:55 - Wisconsin - 8 - 2002 - TAVG
FIPS:55 - Wisconsin - 9 - 2002 - TAVG
FIPS:55 - Wisconsin - 10 - 2002 - TAVG
FIPS:55 - Wisconsin - 11 - 2002 - TAVG
FIPS:55 - Wisconsin - 12 - 2002 - TAVG
FIPS:55 - Wisconsin - 1 - 2003 - TAVG
FIPS:55 - Wisconsin - 2 - 2003 - TAVG
FIPS:55 - Wisconsin - 3 - 2003 - TAVG
FIPS:55 - Wisconsin - 4 - 2003 - TAVG
FIPS:55 - Wisconsin - 5 - 2003 - TAVG
FIPS:55 - Wisconsin - 6 - 2003 - TAVG
FIPS:55 - Wisconsin - 7 - 2003 - TAVG
FIPS:55 - Wisconsin - 8 - 2003 - TAVG
FIPS:55 - Wisconsin - 9 - 2003 - TAVG
FIPS:55 - Wisconsin - 10 - 2003 - TAVG
FIPS:55 - Wisconsin - 11 - 2003 - TAVG
FIPS:55 - Wisconsin - 12 - 2003 - TAVG
FIPS:55 - Wisconsin - 1 - 2004 - TAVG
FIPS:55 - Wisconsin - 2 - 2004 - TAVG
FIPS:55 - Wisconsin - 3 - 2004 - TAVG
FIPS:5

FIPS:56 - Wyoming - 7 - 2009 - TAVG
FIPS:56 - Wyoming - 8 - 2009 - TAVG
FIPS:56 - Wyoming - 9 - 2009 - TAVG
FIPS:56 - Wyoming - 10 - 2009 - TAVG
FIPS:56 - Wyoming - 11 - 2009 - TAVG
FIPS:56 - Wyoming - 12 - 2009 - TAVG
FIPS:56 - Wyoming - 1 - 2010 - TAVG
FIPS:56 - Wyoming - 2 - 2010 - TAVG
FIPS:56 - Wyoming - 3 - 2010 - TAVG
FIPS:56 - Wyoming - 4 - 2010 - TAVG
FIPS:56 - Wyoming - 5 - 2010 - TAVG
FIPS:56 - Wyoming - 6 - 2010 - TAVG
FIPS:56 - Wyoming - 7 - 2010 - TAVG
FIPS:56 - Wyoming - 8 - 2010 - TAVG
FIPS:56 - Wyoming - 9 - 2010 - TAVG
FIPS:56 - Wyoming - 10 - 2010 - TAVG
FIPS:56 - Wyoming - 11 - 2010 - TAVG
FIPS:56 - Wyoming - 12 - 2010 - TAVG


In [37]:
weather_df = pd.DataFrame(weather_data)
weather_df

,Year,Month,State,TAVG
0,2000,1,Alabama,47.156
1,2000,2,Alabama,52.332
2,2000,3,Alabama,58.888
3,2000,4,Alabama,60.844
4,2000,5,Alabama,74.008
...,...,...,...,...
6727,2010,8,Wyoming,66.400
6728,2010,9,Wyoming,57.408
6729,2010,10,Wyoming,48.600
6730,2010,11,Wyoming,29.116


In [2]:
# check and see if we had any failures
faliures

[]

In [41]:
# write the data to a csv
weather_df.to_csv('Resources\temperatures.csv', index=False)